# Import Statements 

In [1]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import math
import time
import datetime
import json
import copy

# Constants

In [2]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'credentials.json'
# VIEW_ID_LIST = ['261530446', '261503383']
VIEW_ID_LIST = ['16422116','24208270','24275372','24275642','24276439','24276570','24276623','24276749','25786802','40101128','48513172','54030308','54938815','69118251','96919288','99976673','126744643','176461949','194489220','194500113','194500244','69118486','198779623','262652040','198797194','198801222','262645691','198758163','198760045','198753112','198778052','198768920','198776252','198776253','198753113','198799306','198785920','47475482','87852214','88697548','88735661','88754415','88930536','88932135','88932136','88934427','88936823','88940123','88941821']
with open(KEY_FILE_LOCATION, 'r') as f:
    KEY_FILE_DATA = json.load(f)

# Google Analytics

In [8]:
def initialize_analyticsreporting():
    """
    Initializes an Analytics Reporting API V4 service object.

    Returns:
        An authorized Analytics Reporting API V4 service object.
    """

#     credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(keyfile_dict = KEY_FILE_DATA, scopes = SCOPES)
    
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics



def get_response_json(analytics, view_id, pageTokenValue,pageSizeValue,metricsValue,dimensionsValue,includeEmptyRowsValue, start_date, end_date):
    """
    Queries the Analytics Reporting API V4.

    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    pageTokenValue: A string to tell the token (id) of the page
    pageSizeValue: A integer to tell the number of rows on a page
    metricsValue: A list of dictionaries to tell the metrics to be fetched
    dimensionsValue: A list of dictionaries to tell the dimensions to be fetched
    includeEmptyRowsValue: A boolean to tell whether to fetch the empty rows or not
    
    Returns:
    The Analytics Reporting API V4 response.
    """
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': view_id,
          'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
          'metrics': metricsValue,
          'dimensions': dimensionsValue,
          'pageSize': pageSizeValue,
          'pageToken': pageTokenValue,
          'includeEmptyRows': includeEmptyRowsValue           
        }]
      }
    ).execute()


def json_to_df(response):
    """
    Converts the Json reponse of Analytics Reporting API V4 to a pandas dataframe

    Args:
    response: A JSON response from Analytics Reporting API V4
    
    Returns:
    The Analytics Reporting API V4 response converted to a dataframe.
    """
    
    dimensions = response["reports"][0]["columnHeader"]["dimensions"] #list of dimensions
    metrics_dict = response["reports"][0]["columnHeader"]["metricHeader"]["metricHeaderEntries"]
    metrics = [] #list of metrics
    
    for metric in metrics_dict:
        metrics.append(metric["name"])
        
    columnHeaders = dimensions + metrics #list of column headers

    if "rows" in response["reports"][0]["data"]: 
        dataRows = response["reports"][0]["data"]["rows"]
        data = pd.json_normalize(dataRows)
        data_dimensions = pd.DataFrame(data['dimensions'].tolist())
        data_metrics = pd.DataFrame(data['metrics'].tolist())
        data_metrics = data_metrics.applymap(lambda x: x['values'])
        data_metrics = pd.DataFrame(data_metrics[0].tolist())
        result = pd.concat([data_dimensions, data_metrics], axis=1, ignore_index=True)
        result.columns = columnHeaders

        return result
    else:
        result = pd.DataFrame(columns = columnHeaders)
        return result
    
def get_next_page_token(responseJson):
    """
    Checks whether the Json reponse of Analytics Reporting API V4 contains the Next Page Token parameter
    The JSON response will contain the Next Page token parameter only if the number of rows exceeds the max allowed rows.

    Args:
    response: A JSON response from Analytics Reporting API V4
    
    Returns:
    If nextPageToken parameter is found - the value of the parameter
    If nextPagetoker parameter is not found - "Not found"
    """
    
    reportContents = responseJson["reports"][0]
    if "nextPageToken" in reportContents:
        return reportContents["nextPageToken"]
    else:
        return "Not found"


def get_df(queryDetails, view_id, start_date, end_date):
    
    analytics = initialize_analyticsreporting()
    nextPageToken = ""
    rowsPerPage = 100000
    response = get_response_json(analytics, view_id, nextPageToken,rowsPerPage,queryDetails["metrics"],queryDetails["dimensions"],True, start_date, end_date)    
    try:
        noOfRows = response["reports"][0]["data"]["rowCount"]
    except KeyError:
        print ("No data returned from Google Analytics")
    else:
        totalPages = math.ceil(int(noOfRows)/rowsPerPage)

        df = None
        df = pd.DataFrame()

        for x in range(1,totalPages + 1):
            time.sleep(2)
            response = get_response_json(analytics, view_id, nextPageToken,rowsPerPage,queryDetails["metrics"],queryDetails["dimensions"],True, start_date, end_date)    
            nextPageToken = get_next_page_token(response)
            newDf = json_to_df(response)
            df = pd.concat([df, newDf], axis = 'rows')
        return df

    
def get_data_mgmt(queryDetails):
    start_date = get_date("2022-05-01")
    end_date = get_date("2022-05-02")

    all_views_df = pd.DataFrame()
    
    for view_id in VIEW_ID_LIST:
        
        start = 0
        end = 10
        query_details_view = copy.deepcopy(queryDetails)
        metrics = query_details_view['metrics']
        no_of_metrics = len(metrics)
        metrics_per_loop = 10
        no_of_loops =  math.ceil(no_of_metrics/metrics_per_loop)
        one_view_df = pd.DataFrame()
        for i in range(no_of_loops):
            print(view_id, i)
            query_details_view['metrics'] = metrics[start:end]
            df_response = get_df(query_details_view, view_id, start_date, end_date)
#             print(view_id, df_response.columns.to_list())
            if one_view_df.empty:
                one_view_df = pd.concat([one_view_df,df_response], axis = 'rows')
            else:
                one_view_df = pd.merge(one_view_df,df_response)
            start = start + metrics_per_loop
            end = end + metrics_per_loop
            


        one_view_df['View'] = view_id
        all_views_df = pd.concat([one_view_df,all_views_df])
        
    print(len(all_views_df), queryDetails['tableName'])
    all_views_df.to_csv(queryDetails['tableName'] + '.csv', index = False)

In [9]:
def get_date(manual_date = "", delta_days = 1, is_iso = False):
    """
    Returns the date in yyyy-mm-dd format
    
    Arguments:
    manual_date (string): Provide the date in yyyy-mm-dd format
    delta_days (int): The number of days to be added or deducted from the date object. 
    """
    if manual_date != "":
        obj_date = datetime.datetime.strptime(manual_date, '%Y-%m-%d').date()
        str_date = str(obj_date)
    else:
        obj_date = datetime.datetime.utcnow().date() #returns today date
        obj_date = obj_date - datetime.timedelta(days = delta_days) #returns date after adding/reducing the delta days
        str_date = str(obj_date)
        

    return str_date

In [10]:
by_device = {'dimensions':[
                              {'name': 'ga:date'},
                              {'name': 'ga:deviceCategory'},
                              {'name': 'ga:channelGrouping'},
                              {'name': 'ga:sourceMedium'},
                             ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
              'tableName': 'ByDevice',
            }
get_data_mgmt(by_device)

16422116 0
24208270 0
No data returned from Google Analytics
24275372 0
No data returned from Google Analytics
24275642 0
No data returned from Google Analytics
24276439 0
No data returned from Google Analytics
24276570 0
No data returned from Google Analytics
24276623 0
No data returned from Google Analytics
24276749 0
No data returned from Google Analytics
25786802 0
40101128 0
No data returned from Google Analytics
48513172 0
54030308 0
54938815 0
No data returned from Google Analytics
69118251 0
96919288 0
No data returned from Google Analytics
99976673 0
No data returned from Google Analytics
126744643 0
176461949 0
194489220 0
194500113 0
194500244 0
69118486 0
198779623 0
262652040 0
198797194 0
No data returned from Google Analytics
198801222 0
262645691 0
198758163 0
No data returned from Google Analytics
198760045 0
No data returned from Google Analytics
198753112 0
No data returned from Google Analytics
198778052 0
No data returned from Google Analytics
198768920 0
No data r

In [ ]:
by_metro_area = {'dimensions':[
                                {'name': 'ga:date'},
                                {'name': 'ga:country'},
                                {'name': 'ga:region'},
                                {'name': 'ga:metro'},
                                {'name': 'ga:city'},
                                {'name': 'ga:channelGrouping'},
                                {'name': 'ga:sourceMedium'},
                                ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
                  'tableName': 'ByMetroArea',
                }
get_data_mgmt(by_metro_area)

In [ ]:
by_age = {'dimensions':[
                        {'name': 'ga:date'},
                        {'name': 'ga:userAgeBracket'},
#                         {'name': 'ga:sourceMedium'},
#                         {'name': 'ga:channelGrouping'},
                        ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
              'tableName': 'ByAge',
         }
        
get_data_mgmt(by_age)

In [ ]:
by_gender = {'dimensions':[
                            {'name': 'ga:date'},
                            {'name': 'ga:userGender'},
#                                 {'name': 'ga:sourceMedium'},
#                                 {'name': 'ga:channelGrouping'},
                            ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
              'tableName': 'ByGender',
            }
get_data_mgmt(by_gender)

In [ ]:
by_keyword = {'dimensions':[
                              {'name': 'ga:date'},
                              {'name': 'ga:keyword'},
                              {'name': 'ga:sourceMedium'},
                              {'name': 'ga:channelGrouping'},
                             ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
              'tableName': 'ByKeyword',
             }
get_data_mgmt(by_keyword)

In [ ]:
by_referral_path = {'dimensions':[
                                  {'name': 'ga:date'},
                                  {'name': 'ga:referralPath'},
                                  {'name': 'ga:sourceMedium'},
                                  {'name': 'ga:channelGrouping'},
                             ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
                     'tableName': 'ByReferralPath',
                   }
get_data_mgmt(by_referral_path)

In [ ]:
by_full_referrer = {'dimensions':[
                              {'name': 'ga:date'},
                              {'name': 'ga:fullReferrer'},
                              {'name': 'ga:sourceMedium'},
                              {'name': 'ga:channelGrouping'},
                             ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
                  'tableName': 'ByFullReferrer',
                   }
get_data_mgmt(by_full_referrer)

In [ ]:
by_social_network = {'dimensions':[
                              {'name': 'ga:date'},
                              {'name': 'ga:socialNetwork'},
                              {'name': 'ga:sourceMedium'},
                              {'name': 'ga:channelGrouping'},
                             ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
                      'tableName': 'BySocialNetwork',
                    }

get_data_mgmt(by_social_network)

In [ ]:
by_date = {'dimensions':[
                        {'name': 'ga:date'},
                        {'name': 'ga:sourceMedium'},
                        {'name': 'ga:channelGrouping'},
                        ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
          'tableName': 'ByDate',
        }

get_data_mgmt(by_date)

In [ ]:
by_pages = {'dimensions':[
                                {'name': 'ga:date'},
                                {'name': 'ga:exitPagePath'},
                                {'name': 'ga:landingPagePath'},
                                {'name': 'ga:pagePath'},
                                {'name': 'ga:sourceMedium'},
                                {'name': 'ga:channelGrouping'},
#                                 {'name': 'ga:PageTitle'},    
                                ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
              'tableName': 'ByPages',
        }
get_data_mgmt(by_pages)

In [ ]:
by_pages_v2 = {'dimensions':[
                                {'name': 'ga:date'},
                                {'name': 'ga:exitPagePath'},
                                {'name': 'ga:landingPagePath'},
                                {'name': 'ga:pagePath'},
                                {'name': 'ga:sourceMedium'},
                                {'name': 'ga:channelGrouping'},
#                                 {'name': 'ga:PageTitle'},    
                                ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
                  'tableName': 'ByPagesV2',
        }
get_data_mgmt(by_pages_v2)

In [ ]:
by_acq_channel = {'dimensions':[
                                {'name': 'ga:date'},
                                {'name': 'ga:source'},
                                {'name': 'ga:medium'},
                                {'name': 'ga:sourceMedium'},
                                {'name': 'ga:channelGrouping'},
                                ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
                  'tableName': 'ByAcquisition Channel',
        }
get_data_mgmt(by_acq_channel)

In [ ]:
by_search_query = {'dimensions':[
                              {'name': 'ga:date'},
                              {'name': 'ga:adMatchedQuery'},
                              {'name': 'ga:sourceMedium'},
                              {'name': 'ga:channelGrouping'},
                             ],
              'metrics':[
                          {'expression': 'ga:transactionRevenue'}, #revenue
                          {'expression': 'ga:revenuePerTransaction'}, #average order value
                          {'expression': 'ga:transactionsPerSession'}, #Ecommerece Conversion Rate
                          {'expression': 'ga:sessions'}, #sessions
                          {'expression': 'ga:users'}, #users
                          {'expression': 'ga:transactions'}, #transactions
                         ],
              'tableName': 'BySearchQuery',
             }
get_data_mgmt(by_search_query)

In [ ]:
by_transaction = {'dimensions':[
                              {'name': 'ga:transactionid'},
#                               {'name': 'ga:date'},
                              {'name': 'ga:clientid'},
                              {'name': 'ga:datehourminute'},
                              {'name': 'ga:ordercouponcode'},
                              {'name': 'ga:shoppingstage'},
                              {'name': 'ga:daystotransaction'},
                        {'name': 'ga:sessionstotransaction'},
#                         {'name': 'ga:productsku'},
#                         {'name': 'ga:productname'},
#                         {'name': 'ga:internalpromotionname'},
#                         {'name': 'ga:internalpromotionid'},
#                         {'name': 'ga:productbrand'},
#                         {'name': 'ga:productcategoryhierarchy'},
#                         {'name': 'ga:productcouponcode'},
#                         {'name': 'ga:productlistname'},
#                         {'name': 'ga:productvariant'},
                             ],
              'metrics':[
                        {'expression': 'ga:transactions'},
                        {'expression': 'ga:transactionspersession'},
                        {'expression': 'ga:transactionrevenue'},
                        {'expression': 'ga:revenuepertransaction'},
                        {'expression': 'ga:transactionrevenuepersession'},
                        {'expression': 'ga:transactionshipping'},
                        {'expression': 'ga:transactiontax'},
                        {'expression': 'ga:totalvalue'},
                        {'expression': 'ga:buytodetailrate'},
                        {'expression': 'ga:carttodetailrate'},
                        {'expression': 'ga:internalpromotionctr'},
                        {'expression': 'ga:internalpromotionclicks'},
                        {'expression': 'ga:internalpromotionviews'},
                        {'expression': 'ga:productcheckouts'},
                        {'expression': 'ga:productdetailviews'},
                        {'expression': 'ga:productlistctr'},
                        {'expression': 'ga:productlistclicks'},
                        {'expression': 'ga:productlistviews'},
                        {'expression': 'ga:productrefundamount'},
                        {'expression': 'ga:productrefunds'},
                        {'expression': 'ga:productremovesfromcart'},
                        {'expression': 'ga:productrevenueperpurchase'},
                        {'expression': 'ga:quantityaddedtocart'},
                        {'expression': 'ga:quantitycheckedout'},
                        {'expression': 'ga:quantityrefunded'},
                        {'expression': 'ga:quantityremovedfromcart'},
                        {'expression': 'ga:productaddstocart'},
#                         {'expression': 'ga:localproductrefundamount'},
#                         {'expression': 'ga:localitemrevenue'},

                         ],
              'tableName': 'ByTransaction',
             }
get_data_mgmt(by_transaction)